In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExTraTrack/rcp_files/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
#        print (i)
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
#            if int(Rows[i][84+l:88+l]) != 2071 and int(Rows[i][84+l:88+l]) != 2072:
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][8+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove -999.0 Data Points
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine DataFrames
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
    if Model == "Control":
        Year_Start = 1900
    elif Model == "RCP45":
        Year_Start = 2000
    elif Model == "RCP85":
        Year_Start = 2100
#        Year_Diff_A = Year_Start - 1985
#        Year_Diff_B = Year_Start - 1985 + 30
#        Year_Diff_C = Year_Start - 1985 + 60
    Origi_List_A = []
    Origi_List_B = []
    Origi_List_C = []
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Kinen = 728
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Kinen = 728
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Kinen = 728
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            if (DF_A["Orig Time(Z)"][k].year != 2071) and (DF_A["Orig Time(Z)"][k].year != 2072):
                List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            if (DF_B["Orig Time(Z)"][k].year != 2071) and (DF_B["Orig Time(Z)"][k].year != 2072):
                List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            if (DF_C["Orig Time(Z)"][k].year != 2071) and (DF_C["Orig Time(Z)"][k].year != 2072):
                List.append(DF_C[Vars[n]][k])
        Array[n] = List
#        Array[n] = numpy.concatenate((DF_A_Var[Vars[n]], DF_B_Var[Vars[n]], DF_C_Var[Vars[n]]))
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
# Ignore Orig Years of 2071-2072 Due to Data Corrupsion
    if Orig_Year < 2071 or Orig_Year >= 2073:
        if Orig_Year < 2071:
            Time_Year = Orig_Year + Year_Diff_A
        elif Orig_Year >= 2073:
            Time_Year = Orig_Year + Year_Diff_A - 2
        if Time_Year == Year:
            Count += 1
        else:
            Year += 1
            Count = 1
        if Count < 10:
            Code = "TC"+str(Year)+"0"+str(Count)
        else:
            Code = "TC"+str(Year)+str(Count)
        return (Code, Year, Count)

In [6]:
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
def Update_Year(Orig_Time, Year_Diff):
    if Orig_Time.year < 2071:
        Year_Update = Orig_Time.year + Year_Diff
    elif Orig_Time.year >= 2073:
        Year_Update = Orig_Time.year + Year_Diff - 2
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [9]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes, DF):
    Storm_Names = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Names.append(Name)
    Codes["Name"] = Storm_Names
    DF_Names = []
    for k in range(len(DF)):
        Code = DF["Code"][k]
        Name = list(Codes[Codes["New Code"] == Code]["Name"])[0]
        DF_Names.append(Name)
    DF.insert(1, "Name", DF_Names)
    return (Codes, DF)

In [10]:
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [11]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [12]:
Name_List = Open_Name_List('Storm_Name_List.csv')

In [13]:
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_B_DF_Orig, Control_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.003_avg')
Control_C_DF_Orig, Control_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')

In [14]:
Control_DF_Combined, Control_Codes, Control_Year_Diffs = \
Combine_DF(Control_A_DF_Orig, Control_B_DF_Orig, Control_C_DF_Orig, \
Control_A_Storm_List_Orig, Control_B_Storm_List_Orig, Control_C_Storm_List_Orig, "Control")
Control_Codes_Names, Control_DF_Names = Assign_Name(Name_List, Control_Codes, Control_DF_Combined)

In [15]:
RCP45_A_DF_Orig, RCP45_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45_avg')
RCP45_B_DF_Orig, RCP45_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg')
RCP45_C_DF_Orig, RCP45_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg')

In [16]:
RCP45_DF_Combined, RCP45_Codes, RCP45_Year_Diffs = \
Combine_DF(RCP45_A_DF_Orig, RCP45_B_DF_Orig, RCP45_C_DF_Orig, \
RCP45_A_Storm_List_Orig, RCP45_B_Storm_List_Orig, RCP45_C_Storm_List_Orig, "RCP45")
RCP45_Codes_Names, RCP45_DF_Names = Assign_Name(Name_List, RCP45_Codes, RCP45_DF_Combined)

In [17]:
RCP85_A_DF_Orig, RCP85_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85_avg')
RCP85_B_DF_Orig, RCP85_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.002_avg')
RCP85_C_DF_Orig, RCP85_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg')

In [18]:
RCP85_DF_Combined, RCP85_Codes, RCP85_Year_Diffs = \
Combine_DF(RCP85_A_DF_Orig, RCP85_B_DF_Orig, RCP85_C_DF_Orig, \
RCP85_A_Storm_List_Orig, RCP85_B_Storm_List_Orig, RCP85_C_Storm_List_Orig, "RCP85")
RCP85_Codes_Names, RCP85_DF_Names = Assign_Name(Name_List, RCP85_Codes, RCP85_DF_Combined)

In [19]:
# Check For Repeated Entries
def Repeated_Check(DF):
    for i in range(len(DF)):
        Lat = DF["Lat"][i]
        Lon = DF["Lon"][i]
        Time = DF["Time(Z)"][i]
        DF_Repeat = DF[(DF["Lat"] == Lat) & (DF["Lon"] == Lon) & (DF["Time(Z)"] == Time)]
        if len(DF_Repeat) > 1:
            print (DF["Code"][i], Time)

In [20]:
#Codes = RCP85_Codes_Names
#Codes[Codes["New Code"] == "TC211505"]

In [21]:
#DF_Storm = Find_Storm(RCP85_DF_Names, "TC201904")
#DF_Storm

In [22]:
#DF_Storm = Find_Storm(RCP85_DF_Names, "TC201905")
#DF_Storm

In [23]:
#Repeated_Check(RCP85_DF_Names)

In [24]:
# Create DF With Information of Each Storm
def Storm_Info_DF(DF, Codes):
    Storm_Types = []
    Path_Types = []
    Start_Times = []
    End_Times = []
    Track_Discons = []
    for i in range(len(Codes)):
        DF_Storm_Orig = DF[DF["Code"] == Codes["New Code"][i]].reset_index()
        DF_Storm, Track_Discons = DF_Fixing(DF_Storm_Orig, Track_Discons)
        Storm_Type, Trans_Type, Start_Time_k, End_Time_k = ET_Function(DF_Storm)
#        print (Storm_Type, Start_Time_k, End_Time_k)
        Start_Time = DF_Storm["Time(Z)"][Start_Time_k]
        if End_Time_k > 0:
            End_Time = DF_Storm["Time(Z)"][End_Time_k]
        else:
            End_Time = numpy.nan
        Start_Times.append(Start_Time)
        End_Times.append(End_Time)
        Storm_Types.append(Storm_Type)
        Path_Types.append(Trans_Type)
        if i == 0:
            DF_Fixed = DF_Storm.copy()
        else:
            DF_Fixed = pandas.concat([DF_Fixed, DF_Storm])
#
# Create Storms DF
    Storms_DF_Orig = pandas.DataFrame({"Code": Codes["New Code"], "Name": Codes["Name"], \
    "Storm Type": Storm_Types, "Path Type": Path_Types, "Start Time": Start_Times, "End Time": End_Times})
    return (DF_Fixed, Storms_DF_Orig)

In [25]:
# Check For Time Discontinuity
def Time_Discontinuity(DF_Storm):
    Time_Continuous = 0
#    print (DF_Storm["Code"][0], len(DF_Storm))
    for k in range(len(DF_Storm)):
        if k > 0 and Time_Continuous == 0:
#            print (k)
# If Over 12 Hours of Missing Data
            if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] > datetime.timedelta(hours=12):
#                print (DF_Storm["Code"][0], k, DF_Storm["Time(Z)"][k], DF_Storm["Time(Z)"][k-1])
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Time_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
    if Time_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated)

In [26]:
# Check For Track Discontinuity
def Track_Discontinuity(DF_Storm):
    Track_Continuous = 0
    Track_Discon = []
    for k in range(len(DF_Storm)):
        if k > 1 and Track_Continuous == 0:
# Calculate Distance Between Each 6 Hour Interval
            Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
            DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
            Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
            DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 500km and Distance Over 5 Times Further Than Previous Time Interval
            if Distance_1 >= 500 and Distance_1 >= Distance_2 * 5:
                Track_Discon = [DF_Storm["Code"][0], DF_Storm["Time(Z)"][k]]
#                print (DF_Storm["Code"][0], k, Distance_1, Distance_2, DF_Storm["Time(Z)"][k])
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Track_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
    if Track_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated, Track_Discon)

In [27]:
# Fix Storms That Have Time Discontinuity or Track Discontinuity
def DF_Fixing(DF_Storm_Orig, Track_Discons):
    Bottom_Right = False
    Top_Right = False
    Bottom_Left = False
    Top_Left = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    ET_Complete = False
#
# Check For Time Discontinuity
    DF_Storm = Time_Discontinuity(DF_Storm_Orig)
# Repeat to Check Again
    DF_Storm = Time_Discontinuity(DF_Storm)
    DF_Storm = Time_Discontinuity(DF_Storm)
#
# Check If Track Continous
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
# Repeat to Check Again
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    return (DF_Storm, Track_Discons)

In [28]:
# Function to Find Whether a Storm Has Completed ET, and Start and End Time of ET
def ET_Function(DF_Storm):
    Tropical_Begin = False
    ET_Begin = False
    ET_Complete = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    Start_Time_k = 0
    End_Time_k = 0
    Storm_Type = 0
    Trans_Type = 0
    for k in range(len(DF_Storm)):
        if ET_Complete == False:
            if Tropical_Begin == False:
# Storm Must First Be Tropical
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                    Tropical_Begin = True
                    Bottom_Right_k.append(k)
            if Tropical_Begin == True:
# Storm Must Be Tropical For At Least 48 Hours
                if len(Bottom_Right_k) <= 8:
                    if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                        Bottom_Right_k.append(k)
                else:
                    if ET_Begin == False:
                        if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                            ET_Begin = True
                            Top_Right_k.append(k)
                        elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                            ET_Begin = True
                            Bottom_Left_k.append(k)
# If Storm Directly Enters Top Left Quadrant = Completed ET
                        elif DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                            ET_Complete = True
                            Start_Time_k = k
                            End_Time_k = k
                            Trans_Type = 3
                    elif ET_Begin == True:
                        if len(Top_Right_k) > 0:
                            if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
# Storm Needs to Stay Within Top Right Quadrant For At Least 24 Hours
                                if len(Top_Right_k) == 4:
                                    Start_Time_k = Top_Right_k[0]
# If Storm Enters Bottom Left Quadrant Within 24 Hours After Leaving Top Right Quadrant = Completed ET
                            elif DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                Start_Time_k = Top_Right_k[0]
                                End_Time_k = k
                                Trans_Type = 1
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# Reset the Counting if Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Top_Right_k[len(Top_Right_k)-1]:
                                    Top_Right_k = []
                        elif len(Bottom_Left_k) > 0:
                            if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                                Bottom_Left_k.append(k)
# Storm Needs to Stay Within Bottom Left Quadrant For At Least 24 Hours
                                if len(Bottom_Left_k) == 4:
                                    Start_Time_k = Bottom_Left_k[0]
# If Storm Enters Top Right Quadrant Within 24 Hours After Leaving Top Right Quadrant = Completed ET
                            elif DF_Storm["B"][k] > 15:
                                ET_Complete = True
                                Start_Time_k = Bottom_Left_k[0]
                                End_Time_k = k
                                Trans_Type = 2
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# Reset the Counting if Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Bottom_Left_k[len(Bottom_Left_k)-1]:
                                    Bottom_Left_k = []
                        elif len(Top_Right_k) == 0 and len(Bottom_Left_k) == 0:
                            if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                                Bottom_Left_k.append(k)
                            elif DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                Start_Time_k = k
                                End_Time_k = k
                                Trans_Type = 3
#
# If ET Never Begun:
    if ET_Complete == False:
        Trans_Type = 0
        if ET_Begin == False:
            Start_Time_k = len(DF_Storm)-1
            End_Time_k = -728
            Storm_Type = "No ET Transition"
# If ET Began But Did Not Complete
        elif ET_Begin == True:
            End_Time_k = -728
            Storm_Type = "ET Transition Incomplete"
# If ET Completed
    else:
        Storm_Type = "ET Transition Complete"
    return (Storm_Type, Trans_Type, Start_Time_k, End_Time_k)

In [29]:
Control_DF_Fixed, Control_Storms_DF_Orig = Storm_Info_DF(Control_DF_Names, Control_Codes_Names)

In [30]:
RCP45_DF_Fixed, RCP45_Storms_DF_Orig = Storm_Info_DF(RCP45_DF_Names, RCP45_Codes_Names)

In [31]:
RCP85_DF_Fixed, RCP85_Storms_DF_Orig = Storm_Info_DF(RCP85_DF_Names, RCP85_Codes_Names)

In [32]:
# Create ET DF From Storms DF With Just Storms That Completed ET Transition
def ET_DF_Filter(Main_DF, Storms_DF_Orig, Codes_DF):
    ET_DF = Storms_DF_Orig[Storms_DF_Orig["Storm Type"] == "ET Transition Complete"].reset_index()
    ET_DF = ET_DF.drop("index", axis=1)
    ET_DF = ET_DF.drop("Storm Type", axis=1)
    Storms_DF = Storms_DF_Orig.copy()
#
# Filter For Just Storms That Completed ET Transition
    Code_List = ET_DF["Code"]
    Filtered_DF = Main_DF[Main_DF["Code"].isin(Code_List)].reset_index()
    Filtered_Codes = Codes_DF[Codes_DF["New Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    Filtered_Codes = Filtered_Codes.drop("index", axis=1)
    return (ET_DF, Storms_DF, Filtered_DF, Filtered_Codes)

In [33]:
Control_ET_Orig, Control_Storms_Final, Control_DF_Filtered, Control_Codes_Final = \
ET_DF_Filter(Control_DF_Fixed, Control_Storms_DF_Orig, Control_Codes_Names)
RCP45_ET_Orig, RCP45_Storms_Final, RCP45_DF_Filtered, RCP45_Codes_Final = \
ET_DF_Filter(RCP45_DF_Fixed, RCP45_Storms_DF_Orig, RCP45_Codes_Names)
RCP85_ET_Orig, RCP85_Storms_Final, RCP85_DF_Filtered, RCP85_Codes_Final = \
ET_DF_Filter(RCP85_DF_Fixed, RCP85_Storms_DF_Orig, RCP85_Codes_Names)

In [34]:
def Storm_Phase_Info(Main_DF, ET_DF):
    Storm_Phase_List = []
    for i in range(len(ET_DF)):
        DF_Storm = Main_DF[Main_DF["Code"] == ET_DF["Code"][i]].reset_index()
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm_Phase = Find_Storm_Phase(DF_Storm, Start_Time, End_Time)
        for j in range(len(Storm_Phase)):
            Storm_Phase_List.append(Storm_Phase[j])
    Main_DF["Storm Phase"] = Storm_Phase_List
    Main_DF = Main_DF.drop("level_0", axis=1)
    return (Main_DF)

In [35]:
def Find_Storm_Phase(DF_Storm, Start_Time, End_Time):
    Time = list(DF_Storm["Time(Z)"])
    B = list(DF_Storm["B"])
    VLT = list(DF_Storm["VLT"])
    Storm_Phase = []
    for k in range(len(DF_Storm)):
        if Time[k] < Start_Time:
            Storm_Phase.append("Tropical")
#            if B[k] < 30 and VLT[k] > -100:
#                    ET_Type.append("Tropical")
#            else:
#                ET_Type.append("Transition")
        elif Time[k] < End_Time:
            Storm_Phase.append("Transition")
#            if B[k] > 0 or VLT[k] < 100:
#                ET_Type.append("Transition")
#            else:
#                ET_Type.append("Tropical")
        else:
            Storm_Phase.append("Extratropical")
#            if B[k] > 0 and VLT[k] < 100:
#                ET_Type.append("Extratropical")
#            else:
#                ET_Type.append("Transition")
    return (Storm_Phase)

In [36]:
Control_DF_Final = Storm_Phase_Info(Control_DF_Filtered, Control_ET_Orig)
RCP45_DF_Final = Storm_Phase_Info(RCP45_DF_Filtered, RCP45_ET_Orig)
RCP85_DF_Final = Storm_Phase_Info(RCP85_DF_Filtered, RCP85_ET_Orig)

In [37]:
# Add ET Transition Lat Lon Into ET DataFrame
def ET_Lat_Lon(Main_DF, ET_DF):
    Array = numpy.zeros((7, len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Array[0][i] = numpy.min(Storm["SLP(hPa)"])
        Array[1][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["SLP(hPa)"].iloc[0])
        Array[2][i] = float(Storm[Storm["Time(Z)"] == End_Time]["SLP(hPa)"].iloc[0])
        Array[3][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lon"].iloc[0])
        Array[4][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lat"].iloc[0])
        Array[5][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lon"].iloc[0])
        Array[6][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lat"].iloc[0])
    ET_DF["Min SLP"] = Array[0]
    ET_DF["Start SLP"] = Array[1]
    ET_DF["End SLP"] = Array[2]
    ET_DF["Start Lon"] = Array[3]
    ET_DF["Start Lat"] = Array[4]
    ET_DF["End Lon"] = Array[5]
    ET_DF["End Lat"] = Array[6]
    return (ET_DF)

In [38]:
# Calculate Distance Between ET Start and End Points
def ET_Distance(ET_DF):
    Array = numpy.zeros(len(ET_DF))
    for i in range(len(ET_DF)):
        Array[i] = Find_Distance(ET_DF["Start Lat"][i], ET_DF["End Lat"][i], \
        ET_DF["Start Lon"][i], ET_DF["End Lon"][i])
    ET_DF["ET Dist (km)"] = Array
    return (ET_DF)

In [39]:
# Calculate Duration Between Storm Formation and Start of ET Transition
def ET_Durations(Main_DF, ET_DF):
    Array = numpy.zeros((2,len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Birth_Time = list(Storm["Time(Z)"])[0]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Trop_Time_Diff = Start_Time - Birth_Time
        Trop_Hours = int(Trop_Time_Diff.total_seconds()) / 3600
        Array[0][i] = Trop_Hours
        ET_Time_Diff = End_Time - Start_Time
        ET_Hours = int(ET_Time_Diff.total_seconds()) / 3600
        Array[1][i] = ET_Hours
    ET_DF["ET Duration (hr)"] = Array[1]
    ET_DF["Trop Duration (hr)"] = Array[0]
    return (ET_DF)

In [40]:
def ET_Variables(Final_DF, ET_DF):
    ET_DF = ET_Lat_Lon(Final_DF, ET_DF)
    ET_DF = ET_Distance(ET_DF)
    ET_DF = ET_Durations(Final_DF, ET_DF)
    return (ET_DF)

In [41]:
Control_ET_Final = ET_Variables(Control_DF_Final, Control_ET_Orig)
RCP45_ET_Final = ET_Variables(RCP45_DF_Final, RCP45_ET_Orig)
RCP85_ET_Final = ET_Variables(RCP85_DF_Final, RCP85_ET_Orig)

In [42]:
Control_ET_Final[5:10]

,Code,Name,Path Type,Start Time,End Time,Min SLP,Start SLP,End SLP,Start Lon,Start Lat,End Lon,End Lat,ET Dist (km),ET Duration (hr),Trop Duration (hr)
5,TC190105,Lydia,1,1901-09-27 12:00:00,1901-09-29 18:00:00,913.33,953.76,970.00,-53.37,39.19,-42.82,48.53,1335.846025,54.0,384.0
6,TC190106,Mario,1,1901-09-22 00:00:00,1901-09-22 18:00:00,988.80,993.69,991.98,-78.47,35.78,-71.20,41.68,909.539822,18.0,72.0
7,TC190107,Nicole,2,1901-10-14 12:00:00,1901-10-15 00:00:00,974.56,1015.94,1019.52,-22.00,42.00,-26.00,41.25,343.074813,12.0,246.0
8,TC190109,Shannon,1,1901-12-14 00:00:00,1901-12-14 06:00:00,942.03,975.20,975.97,-32.75,47.00,-30.25,47.75,206.101857,6.0,90.0
9,TC190201,Tony,2,1902-01-27 18:00:00,1902-01-28 00:00:00,1002.71,1012.65,1014.76,-43.50,29.50,-44.50,28.00,193.395015,6.0,60.0


In [43]:
Control_Storms_Final[8:13]

,Code,Name,Storm Type,Path Type,Start Time,End Time
8,TC190104,Kinen,ET Transition Incomplete,0,1901-09-11 00:00:00,NaT
9,TC190105,Lydia,ET Transition Complete,1,1901-09-27 12:00:00,1901-09-29 18:00:00
10,TC190106,Mario,ET Transition Complete,1,1901-09-22 00:00:00,1901-09-22 18:00:00
11,TC190107,Nicole,ET Transition Complete,2,1901-10-14 12:00:00,1901-10-15 00:00:00
12,TC190108,Phil,No ET Transition,0,1901-10-17 18:00:00,NaT


In [44]:
Control_DF_Final[265:277]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Storm Phase
265,TC190105,Lydia,-56.33,37.40,948.48,41.5,9.05,236.10,289.92,1901-09-27 00:00:00,Tropical
266,TC190105,Lydia,-54.83,38.25,950.98,39.9,11.90,234.39,267.61,1901-09-27 06:00:00,Tropical
267,TC190105,Lydia,-53.37,39.19,953.76,43.1,18.13,224.89,230.05,1901-09-27 12:00:00,Transition
268,TC190105,Lydia,-51.56,40.08,952.91,42.0,31.98,231.25,195.68,1901-09-27 18:00:00,Transition
269,TC190105,Lydia,-49.70,42.10,951.29,38.4,43.81,253.98,172.97,1901-09-28 00:00:00,Transition
270,TC190105,Lydia,-49.51,44.55,948.89,44.0,41.29,241.12,161.34,1901-09-28 06:00:00,Transition
271,TC190105,Lydia,-49.52,44.98,949.44,43.2,38.45,220.08,156.22,1901-09-28 12:00:00,Transition
272,TC190105,Lydia,-48.37,45.66,952.82,37.7,28.80,201.31,169.85,1901-09-28 18:00:00,Transition
273,TC190105,Lydia,-47.67,46.28,959.65,33.9,14.50,153.80,168.18,1901-09-29 00:00:00,Transition
274,TC190105,Lydia,-46.92,46.83,963.87,31.0,4.41,76.01,151.33,1901-09-29 06:00:00,Transition


In [45]:
Control_Codes_Final[5:10]

,ABC,Orig Code,New Code,Name
5,A,0089,TC190105,Lydia
6,A,0090,TC190106,Mario
7,A,0095,TC190107,Nicole
8,A,0108,TC190109,Shannon
9,A,0116,TC190201,Tony


In [46]:
# Output File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output_V3.csv')
    Output_Diri = '/glade/u/home/whimkao//ExtraTrack/Output_Files/'
    DF.to_csv(Output_Diri+File_Name)

In [47]:
Output_File(Control_DF_Final, "Control", "Data")
Output_File(Control_ET_Final, "Control", "ET")
Output_File(Control_Codes_Final, "Control", "Codes")
Output_File(Control_Storms_Final, "Control", "Storms")

In [48]:
Output_File(RCP45_DF_Final, "RCP45", "Data")
Output_File(RCP45_ET_Final, "RCP45", "ET")
Output_File(RCP45_Codes_Final, "RCP45", "Codes")
Output_File(RCP45_Storms_Final, "RCP45", "Storms")

In [49]:
Output_File(RCP85_DF_Final, "RCP85", "Data")
Output_File(RCP85_ET_Final, "RCP85", "ET")
Output_File(RCP85_Codes_Final, "RCP85", "Codes")
Output_File(RCP85_Storms_Final, "RCP85", "Storms")

In [50]:
# Some Stuff I've Tried Out that Didn't Work Out...

In [51]:
#DF_Storm = RCP85_DF_Names[RCP85_DF_Names["Code"] == "TC213713"].reset_index()
#DF_Filter_Process(DF_Storm)

In [52]:
#DF_Storm = Find_Storm(RCP45_DF_Names, "TC200409")
#DF_Storm

In [53]:
#RCP45_DF_Names[RCP45_DF_Names["Code"] == "TC200409"]
#RCP45_DF_Combined[1380:1420]

In [54]:
# Examine Each Storm For Any Data Quality Issues
#def Storm_Examination(DF_Orig, Codes):
#    for i in range(len(Codes)):
#        DF_Storm = Find_Storm(DF_Orig, Codes["New Code"][i])
#        for k in range(len(DF_Storm)):
#            if k >= 1:
# First Check For Time Discontinuity
#                if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] > datetime.timedelta(hours=12):
# Find If Any Other Storms at the Same Time
#                    Time_Before = DF_Storm["Time(Z)"][k-1]
#                    Time_Before_Find_Storm(DF_Orig, DF_Storm, Time_Before)
#                    Time_After = DF_Storm["Time(Z)"][k]
#                    Time_After_Find_Storm(DF_Orig, DF_Storm, Time_After)
# Then Check For Track Discontinuity
#            if k >= 2:
#                Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
#                DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
#                Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
#                DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 500km and Distance Over 5 Times Further Than Previous Time Interval
#                if Distance_1 >= 500 and Distance_1 >= Distance_2 * 5:
# Find If Any Other Storms at the Same Time
#                    Time_Before = DF_Storm["Time(Z)"][k-1]
#                    Time_Before_Find_Storm(DF_Orig, DF_Storm, Time_Before)
#                    Time_After = DF_Storm["Time(Z)"][k]
#                    Time_After_Find_Storm(DF_Orig, DF_Storm, Time_After)
#                    
#        if i == 0:
#            DF_Final = DF_Storm.copy()
#        else:
#            DF_Final = pandas.concat([DF_Final, DF_Storm])
        

In [55]:
#Control_Codes[0:640]

In [56]:
#Storm_Examination(Control_DF_Combined[0:23074], Control_Codes[0:640])

In [57]:
#def Time_Before_Find_Storm(DF_Orig, DF_Storm, Time_Before):
#    Found_Anything = False
#    Time = Time_Before + datetime.timedelta(hours=6)
#    DF_Time = DF_Orig[DF_Orig["Time(Z)"] == Time].reset_index()
#    DF_Time_Codes = list(DF_Time["Code"])
#    if DF_Storm["Code"][0] in DF_Time_Codes:
#        DF_Time_Codes.remove(DF_Storm["Code"][0])
#    if len(DF_Time_Codes) > 0:
#        for n in range(len(DF_Time_Codes)):
#            if Check_Distance_SLP(Time_Before, Time, DF_Storm, Find_Storm(DF_Orig, DF_Time_Codes[n])) == True:
#                Found_Anything = True
#                DF_Storm_2 = Find_Storm(DF_Orig, DF_Time_Codes[n])
#                Storm_1_Before = DF_Storm[DF_Storm["Time(Z)"] < Time]
#                Storm_1_After = DF_Storm[DF_Storm["Time(Z)"] >= Time]
#                Storm_2_Before = DF_Storm_2[DF_Storm_2["Time(Z)"] < Time]
#                Storm_2_After = DF_Storm_2[DF_Storm_2["Time(Z)"] >= Time]
#                if len(Storm_2_Before) > 0:
#                    Storm_1_Min_SLP = numpy.min(Storm_1_Before["SLP(hPa)"])
#                    Storm_2_Min_SLP = numpy.min(Storm_2_Before["SLP(hPa)"])
#                    if Storm_2_Min_SLP <= Storm_1_Min_SLP:
#                        New_Storm_1 = Storm_1_After
#                        New_Storm_2 = DF_Storm_2
#                    else:
#                        New_Storm_1 = pandas.concat([Storm_1_Before, Storm_2_After])
#                        New_Storm_2 = Storm_1_After
#                else:
#                    New_Storm_1 = pandas.concat([Storm_1_Before, Storm_2_After])
#                    New_Storm_2 = Storm_1_After
#    if Found_Anything == False:
#        New_Storm_1 = 0
#        New_Storm_2 = 0
#    return (Found_Anything, New_Storm_1, New_Storm_2)

In [58]:
#def Time_After_Find_Storm(DF_Orig, DF_Storm, Time_After):
#    Found_Anything = False
#    Time = Time_After - datetime.timedelta(hours=6)
#    DF_Time = DF_Orig[DF_Orig["Time(Z)"] == Time].reset_index()
#    DF_Time_Codes = list(DF_Time["Code"])
#    if DF_Storm["Code"][0] in DF_Time_Codes:
#        DF_Time_Codes.remove(DF_Storm["Code"][0])
#    if len(DF_Time_Codes) > 0:
#        for n in range(len(DF_Time_Codes)):
#            if Check_Distance_SLP(Time_After, Time, DF_Storm, Find_Storm(DF_Orig, DF_Time_Codes[n])) == True:
#                print ("Yan")
#                Found_Anything = True
#                DF_Storm_2 = Find_Storm(DF_Orig, DF_Time_Codes[n])
#                Storm_1_Before = DF_Storm[DF_Storm["Time(Z)"] <= Time]
#                Storm_1_After = DF_Storm[DF_Storm["Time(Z)"] > Time]
#                Storm_2_Before = DF_Storm_2[DF_Storm_2["Time(Z)"] <= Time]
#                Storm_2_After = DF_Storm_2[DF_Storm_2["Time(Z)"] > Time]
#                if len(Storm_2_Before) > 0:
#                    Storm_1_Min_SLP = numpy.min(Storm_1_After["SLP(hPa)"])
#                    Storm_2_Min_SLP = numpy.min(Storm_2_After["SLP(hPa)"])
#                    if Storm_2_Min_SLP <= Storm_1_Min_SLP:
#                        New_Storm_1 = Storm_1_Before
#                        New_Storm_2 = DF_Storm_2
#                    else:
#                        New_Storm_2 = pandas.concat([Storm_2_Before, Storm_1_After])
#                        New_Storm_1 = Storm_1_Before
#                else:
#                    New_Storm_2 = pandas.concat([Storm_2_Before, Storm_1_After])
#                    New_Storm_1 = Storm_1_Before
#    if Found_Anything == False:
#        New_Storm_1 = 0
#        New_Storm_2 = 0
#    return (Found_Anything, New_Storm_1, New_Storm_2)

In [59]:
#def Check_Distance_SLP(Time_1, Time_2, DF_Storm_1, DF_Storm_2):
#    k = numpy.where(DF_Storm_1["Time(Z)"] == Time_1)[0][0]
#    l = numpy.where(DF_Storm_2["Time(Z)"] == Time_2)[0][0]
#    Distance_Between = Find_Distance(DF_Storm["Lat"][k], DF_Storm_2["Lat"][l], 
#    DF_Storm["Lon"][k], DF_Storm_2["Lon"][l])
#    SLP_Diff = DF_Storm["SLP(hPa)"][k] - DF_Storm_2["SLP(hPa)"][l]
#    if Distance_Between <= 250 and numpy.abs(SLP_Diff) < 15:
#        Result = True
#    else:
#        Result = False
#    return (Result)